# Hate Speech Detector - EN - Features extraction for SVM & Dense model

Based on [this notebook](https://github.com/t-davidson/hate-speech-and-offensive-language/blob/master/classifier/final_classifier.ipynb).

In [1]:
import os
import pandas as pd
import csv
import numpy as np
import pickle
from klepto.archives import dir_archive
import sys
import nltk
import string
import re
import fasttext
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
import syllables as sylla
from pymagnitude import *

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

from tqdm.notebook import tqdm
%matplotlib inline

In [2]:
MODEL = 'dense'
dim = 10 if MODEL == 'svm' else 200

## Davidson et al. data loading

### Classes pre:
    0 - hate speech
    1 - offensive language
    2 - neither

In [3]:
if not os.path.exists('hsd/Reddit/perfect_data.pkl'):
    tweets, labels = [], []
    with open('hsd/Reddit/labeled_data.csv', 'r') as f:
        for d in tqdm(list(csv.reader(f))[1:]):
            tweets.append(d[6])  # tweet
            labels.append(d[5])  # class
    with open('hsd/Reddit/perfect_data.pkl', 'w') as f:
        def chcl(c):
            return 0 if c=='0' else 1
        labels = list(map(chcl, labels))
        pickle.dump((tweets, labels), f)
else:
    with open('hsd/Reddit/perfect_data.pkl', 'rb') as f:
        tweets, labels = pickle.load(f)

### Classes post:
    0 - no hate
    1 - hate speech

In [4]:
print('Tweets: {}'.format(len(tweets)))
print('Labels: {}'.format(len(labels)))

Tweets: 55132
Labels: 55132


In [5]:
list(zip(tweets[:5], labels[:5]))

[("i joined gab to remind myself how retarded jew haters are. You wouldn't be typing on your abacus without them you retard.",
  1),
 ('This is what the left is really scared of.    https://redd.it/9rfkts', 0),
 ('That literally looks like a monkey. Why are we supposed to pretend it’s a person bc it’s wearing a red hat?',
  0),
 ('Dumb Cunt', 1),
 ('It makes you an asshole.', 0)]

## Features extraction

In [6]:
stopwords=stopwords = nltk.corpus.stopwords.words("english")

other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)

sentiment_analyzer = VS()

In [7]:
def simple_preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    #hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    #parsed_text = re.sub(hashtag_regex, '', parsed_text)
    return parsed_text

def pos_tagger(nltk_tag): 
    if nltk_tag.startswith('J'): 
        return wordnet.ADJ 
    elif nltk_tag.startswith('V'): 
        return wordnet.VERB 
    elif nltk_tag.startswith('N'): 
        return wordnet.NOUN 
    elif nltk_tag.startswith('R'): 
        return wordnet.ADV 
    else:           
        return None

def word_tokenization(tweet):
    lemmatizer = WordNetLemmatizer() 
    tokens = word_tokenize(tweet)
    words = [word for word in tokens if word.isalpha()]
    # stop_words = set(stopwords.words('english'))
    # words = [w for w in words if not w in stop_words]
    tags = nltk.pos_tag(words)
    # words = [lemmatizer.lemmatize(w[0]) if pos_tagger(w[1]) is None else lemmatizer.lemmatize(w[0], pos_tagger(w[1])) for w in tags]
    tags = [x[1] for x in tags]
    return words, tags

def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = text_string.encode('ascii', 'ignore').decode('ascii')
    parsed_text = re.sub(space_pattern, ' ', parsed_text)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    parsed_text = parsed_text.strip('#')
    list_words, tag_list = word_tokenization(parsed_text)
    parsed_text = " ".join(list_words)
    tag_str = ' '.join(tag_list)
    return parsed_text, tag_str

def basic_tokenize(tweet):
    tweet = " ".join(re.split(" ", tweet.lower())).strip()
    return tweet.split()

# def get_pos_string(tweet):
#     text = preprocess(tweet)
#     tokens = word_tokenize(text)
#     tags = nltk.pos_tag(tokens)
#     tag_list = [x[1] for x in tags]
#     tag_str = ' '.join(tag_list)
    
    # return tag_str

def pad_words(words, length):
    if len(words) >= length:
        return words[:length]
    else:
        additional = length - len(words)
        return words + ['EMPTY']*additional

def count_twitter_objs(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned.
    
    Returns counts of urls, mentions, and hashtags.
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, 'URLHERE', parsed_text)
    parsed_text = re.sub(mention_regex, 'MENTIONHERE', parsed_text)
    parsed_text = re.sub(hashtag_regex, 'HASHTAGHERE', parsed_text)
    return(parsed_text.count('URLHERE'),parsed_text.count('MENTIONHERE'),parsed_text.count('HASHTAGHERE'))

def other_features(tweet):
    """This function takes a string and returns a list of features.
    These include Sentiment scores, Text and Readability scores,
    as well as Twitter specific features"""
    sentiment = sentiment_analyzer.polarity_scores(tweet)
    
    words = preprocess(tweet)[0] #Get text only
    
    syllables = sylla.estimate(words)
    num_chars = sum(len(w) for w in words)
    num_chars_total = len(tweet)
    num_terms = len(tweet.split())
    num_words = len(words.split())
    avg_syl = round(float((syllables+0.001))/float(num_words+0.001),4)
    num_unique_terms = len(set(words.split()))
    
    ###Modified FK grade, where avg words per sentence is just num words/1
    FKRA = round(float(0.39 * float(num_words)/1.0) + float(11.8 * avg_syl) - 15.59, 1)
    ##Modified FRE score, where sentence fixed to 1
    FRE = round(206.835 - 1.015*(float(num_words)/1.0) - (84.6*float(avg_syl)), 2)
    
    twitter_objs = count_twitter_objs(tweet)
    retweet = 0 if "rt" in words else 1
    features = [FKRA, FRE, syllables, avg_syl, num_chars, num_chars_total, num_terms, num_words,
                num_unique_terms, sentiment['neg'], sentiment['pos'], sentiment['neu'], sentiment['compound'],
                twitter_objs[2], twitter_objs[1],
                twitter_objs[0], retweet]
    return features

### Supervised fastText wordtokens training

In [8]:
if not os.path.exists('hsd/Reddit/fasttext.ft'):
    with open('hsd/Reddit/fasttext.ft', 'a') as f:
        for t, l in list(zip(tweets, labels)):
            text = preprocess(t)[0]
            if len(text) > 0:
                f.write('__label__{} {}\n'.format(l, text))

# load fasttext model or train & save if none
if os.path.exists('hsd/Reddit/fasttext_{}.bin'.format(MODEL)):
    ft_model = fasttext.load_model('hsd/Reddit/fasttext_{}.bin'.format(MODEL))
else:
    ft_model = fasttext.train_supervised('hsd/Reddit/fasttext.ft',
                                         lr=0.5, epoch=50, wordNgrams=3, dim=dim)
    ft_model.save_model('hsd/Reddit/fasttext_{}.bin'.format(MODEL))

### Wordtoken features

In [9]:
def get_wordtoken_fts(data):
    
    sentences_words = []
    for d in data:
        sentence = preprocess(d)[0]
        sentences_words.append(sentence.split(' '))
    
    opt_length = int(np.median([len(sw) for sw in sentences_words]))
    sentences_words = [pad_words(sw, opt_length) for sw in sentences_words]
    
    ft_vectors = []
    for sw in sentences_words:
        ft_vector = []
        for w in sw:
            ft_vector.extend(ft_model[w])
        ft_vectors.append(ft_vector)
    
    return ft_vectors

In [10]:
wordtoken_features = get_wordtoken_fts(tweets)

In [11]:
wordtoken_features[0]

[-0.020074433,
 -0.0010464381,
 0.024866536,
 -0.022601176,
 -0.0059229187,
 0.04829487,
 0.10509797,
 0.056436967,
 -0.13000314,
 -0.023368329,
 0.05507625,
 0.050443072,
 -0.015017701,
 -0.020066796,
 -0.026433446,
 -0.025757207,
 -0.007738385,
 0.044283252,
 0.027462238,
 -0.049093556,
 0.021676162,
 0.040927622,
 0.029724373,
 0.017214471,
 -0.022925016,
 -0.0058470326,
 -0.04114398,
 0.05169604,
 0.062522255,
 -0.04631921,
 -0.0050944826,
 -0.081099875,
 0.033975657,
 -0.01625956,
 0.04779283,
 0.0018938238,
 0.043410126,
 -0.029346984,
 -0.018147536,
 -0.008108199,
 -0.003423023,
 -0.05781455,
 -0.06602539,
 -0.0012340905,
 -0.0057204165,
 -0.024898654,
 -0.03922726,
 -0.06900807,
 -0.061060507,
 -0.026512388,
 -0.00084784144,
 -0.031567097,
 -0.019074213,
 0.010728149,
 0.035440855,
 0.025398087,
 0.014900015,
 0.0728808,
 0.022989659,
 -0.052744262,
 -0.08789888,
 -0.04611443,
 -0.040348634,
 -0.08190622,
 0.032269992,
 -0.10168228,
 0.026007347,
 -0.028958278,
 0.01420271,
 0.

### Supervised fastText pos training

In [12]:
if not os.path.exists('hsd/Reddit/fasttext_pos.ft'):
    with open('hsd/Reddit/fasttext_pos.ft', 'a') as f:
        for t, l in list(zip(tweets, labels)):
            f.write('__label__{} {}\n'.format(l, preprocess(t)[1]))

# load fasttext pos model or train & save if none
if os.path.exists('hsd/Reddit/fasttext_pos_{}.bin'.format(MODEL)):
    ft_pos_model = fasttext.load_model('hsd/Reddit/fasttext_pos_{}.bin'.format(MODEL))
else:
    ft_pos_model = fasttext.train_supervised('hsd/Reddit/fasttext_pos.ft',
                                             lr=0.5, epoch=50, wordNgrams=3, dim=dim)
    ft_pos_model.save_model('hsd/Reddit/fasttext_pos_{}.bin'.format(MODEL))

### Part of speech (PoS) features

In [13]:
def get_pos_fts(data):

    #Get POS tags for tweets and save as a string
    pos_sentences = []
    for d in data:
        pos_string = preprocess(d)[1]
        pos_sentences.append(pos_string)
        
        
    pos_tags = []
    for ps in pos_sentences:
        pos_tags.append(ps.split(' '))
    
    opt_length = int(np.median([len(pt) for pt in pos_tags]))
    pos_tags = [pad_words(pt, opt_length) for pt in pos_tags]
    
    ft_vectors = []
    for pt in pos_tags:
        ft_vector = []
        for t in pt:
            ft_vector.extend(ft_pos_model[t])
        ft_vectors.append(ft_vector)
    
    return ft_vectors

In [14]:
pos_features = get_pos_fts(tweets)

In [15]:
pos_features[0]

[-0.24737507,
 0.37265313,
 -0.10580574,
 -0.6254099,
 -0.68918836,
 0.23592462,
 -0.3190713,
 -0.14368151,
 0.6848899,
 1.6542645,
 0.0017685906,
 -1.0936574,
 -1.149242,
 -0.24874505,
 0.43608025,
 0.39674857,
 0.36702904,
 0.0762877,
 -0.007873814,
 0.887947,
 -0.24737507,
 0.37265313,
 -0.10580574,
 -0.6254099,
 -0.68918836,
 0.23592462,
 -0.3190713,
 -0.14368151,
 0.6848899,
 1.6542645,
 0.09965002,
 1.1912097,
 1.4800516,
 1.148172,
 -0.93075204,
 -1.0350236,
 -0.95710313,
 -0.5307445,
 -0.245457,
 -1.5423375,
 -0.18846865,
 -0.12492545,
 -0.69818294,
 0.6093615,
 -0.19825664,
 0.07976622,
 -0.04633593,
 0.2905052,
 0.26071504,
 0.84542716,
 0.2638475,
 0.017239993,
 0.28201565,
 0.4577914,
 -0.20165063,
 0.057426896,
 -0.09905324,
 -0.2408881,
 0.6813042,
 -0.49243268,
 0.5184322,
 0.7475772,
 1.2300949,
 -0.28866297,
 -0.3084485,
 0.53250855,
 -0.65286666,
 -0.3846743,
 -0.027212892,
 -1.6446086,
 -0.18099292,
 -0.11273867,
 -0.5382735,
 0.22672129,
 0.15421152,
 -0.5887575,
 0

### Other features

In [19]:
other_features = np.array([other_features(t) for t in tweets])

In [20]:
other_features[:5]

array([[ 1.1200e+01,  5.3760e+01,  3.4000e+01,  1.5454e+00,  1.2100e+02,
         1.2100e+02,  2.2000e+01,  2.2000e+01,  2.2000e+01,  2.4100e-01,
         9.7000e-02,  6.6300e-01, -6.2780e-01,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  1.0000e+00],
       [ 2.3000e+00,  9.4300e+01,  1.1000e+01,  1.2222e+00,  4.3000e+01,
         6.8000e+01,  1.0000e+01,  9.0000e+00,  8.0000e+00,  2.6200e-01,
         0.0000e+00,  7.3800e-01, -4.9270e-01,  0.0000e+00,  0.0000e+00,
         1.0000e+00,  1.0000e+00],
       [ 1.0000e+01,  6.0630e+01,  3.1000e+01,  1.4762e+00,  1.0700e+02,
         1.0700e+02,  2.1000e+01,  2.1000e+01,  1.8000e+01,  6.1000e-02,
         1.0900e-01,  8.3000e-01,  2.7320e-01,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  1.0000e+00],
       [-3.0000e+00,  1.2021e+02,  2.0000e+00,  1.0000e+00,  9.0000e+00,
         9.0000e+00,  2.0000e+00,  2.0000e+00,  2.0000e+00,  1.0000e+00,
         0.0000e+00,  0.0000e+00, -7.5790e-01,  0.0000e+00,  0.0000e+00,
         0.0000e+00

In [21]:
np.array(wordtoken_features).shape

(55132, 190)

In [22]:
np.array(pos_features).shape

(55132, 190)

In [23]:
np.array(other_features).shape

(55132, 17)

### All features and feature names

In [26]:
#Now join them all up
features = np.concatenate([wordtoken_features, pos_features, other_features],axis=1)

In [27]:
features.shape

(55132, 397)

## Save features & labels

In [28]:
archive = dir_archive('hsd/Reddit/X_y_{}'.format(MODEL), {'features': features, 'labels': labels}, serialized=True)
archive.dump()
del archive